2.	Подготовьте sentiment analysis по отзывам (используя  nltk.classify) и оцените, как распределяется sentiment по актерам, режиссерам и жанрам

In [16]:
# в файле diplom2_example.ipynb сделан рабочий пример из статьи в инете - 
# https://www.datacamp.com/tutorial/simplifying-sentiment-analysis-python#rdl

# здесь попробовал на его основе научиться распознавать комедии от драм, 
# что-то получилось, но  пока не понял что.
# Марат, подскажи плз в правильном ли направлении копаю?

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import nltk
from nltk.classify import NaiveBayesClassifier
from nltk.tokenize import WordPunctTokenizer
from nltk.corpus import movie_reviews

In [10]:
df = pd.read_csv('wiki_movie_plots_deduped.csv')
df.head()

,Release Year,Title,Origin/Ethnicity,Director,Cast,Genre,Wiki Page,Plot
0,1901,Kansas Saloon Smashers,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/Kansas_Saloon_Sm...,"A bartender is working at a saloon, serving dr..."
1,1901,Love by the Light of the Moon,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/Love_by_the_Ligh...,"The moon, painted with a smiling face hangs ov..."
2,1901,The Martyred Presidents,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/The_Martyred_Pre...,"The film, just over a minute long, is composed..."
3,1901,"Terrible Teddy, the Grizzly King",American,Unknown,NaN,unknown,"https://en.wikipedia.org/wiki/Terrible_Teddy,_...",Lasting just 61 seconds and consisting of two ...
4,1902,Jack and the Beanstalk,American,"George S. Fleming, Edwin S. Porter",NaN,unknown,https://en.wikipedia.org/wiki/Jack_and_the_Bea...,The earliest known adaptation of the classic f...


In [11]:
df_train = df.loc[df['Genre'] != 'unknown']
df_train.Genre.unique() #2264
# len(df_train) #28803

array(['western', 'comedy', 'short', ...,
       'adventure, romance, fantasy film', 'ero',
       'horror romantic comedy'], dtype=object)

In [82]:
# Делаем тренировочный набор данных
genres = ['drama','comedy','western','horror','action','thriller','adventure','crime','musical']
# genres = ['drama','comedy','western']
my_train = df_train[['Plot','Genre']].loc[df.Genre.isin(genres)]
my_train

,Plot,Genre
6,The film opens with two bandits breaking into ...,western
7,The film is about a family who move to the sub...,comedy
14,Before heading out to a baseball game at a nea...,comedy
15,The plot is that of a black woman going to the...,comedy
16,On a beautiful summer day a father and mother ...,drama
...,...,...
34874,Hasan is a twelve-year-old boy living with his...,drama
34875,"Through the night, three cars carry a small gr...",drama
34876,The film opens with a Senegalese boy named Kha...,drama
34882,"Two musicians, Salih and Gürkan, described the...",comedy


In [83]:
tokenizer = WordPunctTokenizer()
# print(my_train.Plot[6])
print(tokenizer.tokenize(my_train.Plot[7]))
ww = [w.lower() for w in tokenizer.tokenize(df.Plot[i]) for i in df.index]
len(ww)

['The', 'film', 'is', 'about', 'a', 'family', 'who', 'move', 'to', 'the', 'suburbs', ',', 'hoping', 'for', 'a', 'quiet', 'life', '.', 'Things', 'start', 'to', 'go', 'wrong', ',', 'and', 'the', 'wife', 'gets', 'violent', 'and', 'starts', 'throwing', 'crockery', ',', 'leading', 'to', 'her', 'arrest', '.']


7430718

In [84]:
# собираем список документов для тренировки
my_docs = [(list(tokenizer.tokenize(my_train.Plot[id])), my_train.Genre[id]) for id in my_train.index]
len(my_docs)

16000

In [85]:
all_words = nltk.FreqDist(w.lower() for w in tokenizer.tokenize(df.Plot[i]) for i in df.index)
# all_words = nltk.FreqDist(w.lower() for w in ww)
len(all_words)

125

In [86]:
# Define the feature extractor

all_words = nltk.FreqDist(w.lower() for w in tokenizer.tokenize(df.Plot[i]) for i in df.index)
word_features = list(all_words)[:2000]

def document_features(document):
    document_words = set(document)
    features = {}
    for word in word_features:
        features['contains({})'.format(word)] = (word in document_words)
    return features

In [87]:
# Train Naive Bayes classifier
featuresets = [(document_features(d), c) for (d,c) in my_docs]
train_set, test_set = featuresets[:9000], featuresets[9000:]
classifier = nltk.NaiveBayesClassifier.train(train_set)

In [88]:
# Test the classifier
print(nltk.classify.accuracy(classifier, test_set))

0.2692857142857143


In [89]:
# Show the most important features as interpreted by Naive Bayes
classifier.show_most_informative_features(10)
# не понял куда букву n потерял у western

Most Informative Features
          contains(king) = True           advent : wester =     55.0 : 1.0
            contains(to) = False           crime : horror =     25.4 : 1.0
             contains(,) = False           crime : horror =     22.5 : 1.0
        contains(asylum) = True           horror : wester =     19.3 : 1.0
        contains(tunnel) = True           horror : wester =     15.3 : 1.0
        contains(liquid) = True           horror : drama  =     15.3 : 1.0
         contains(brave) = True           wester : comedy =     13.1 : 1.0
             contains(a) = False          wester : horror =     11.2 : 1.0
         contains(stone) = True           advent : wester =     11.0 : 1.0
       contains(cottage) = True           horror : comedy =      9.9 : 1.0
